## Deformation Project

#### Imports

In [ ]:
from src.hartree_fock_library import HartreeFock,HartreeFockVariational,gram_schmidt
from src.hamiltonian_utils import get_twobody_nuclearshell_model,FermiHubbardHamiltonian,SingleParticleState
import numpy as np
import torch
from typing import Dict,List
from src.qml_models import AdaptVQEFermiHubbard
from src.qml_utils.train import Fit
from src.qml_utils.utils import configuration
#from src.qml_models import AdaptVQEFermiHubbard
import matplotlib.pyplot as plt
#from adapt_vqe_old import AdaptVQEFermiHubbard

#### The parameters

In [ ]:
twobody_matrix,_,_,energies=get_twobody_nuclearshell_model(file_name='data/cki')

SPS=SingleParticleState(file_name='data/cki')

alfa=1.
size_a=energies.shape[0]//2
size_b=energies.shape[0]//2
nparticles_a=2
nparticles_b=2

#### The Hamiltonian

In [ ]:


#%% initialize the FH Hamiltonian

FHHamiltonian=FermiHubbardHamiltonian(size_a=size_b,size_b=size_a,nparticles_a=nparticles_a,nparticles_b=nparticles_b,symmetries=None)


print(FHHamiltonian.basis.shape)



FHHamiltonian.get_external_potential(external_potential=energies)
FHHamiltonian.get_twobody_interaction(twobody_dict=twobody_matrix)
FHHamiltonian.twobody_operator=alfa*FHHamiltonian.twobody_operator
FHHamiltonian.get_hamiltonian()

es,psi0=FHHamiltonian.get_spectrum(n_states=20)
egs=es[0]
e1st=es[1]
print(egs,e1st)


#### The mutual info

In [ ]:
mutual_info=FHHamiltonian.mutual_info(psi=psi0)

In [ ]:
plt.imshow(mutual_info)
plt.colorbar()
plt.show()